# Web scraper for `tauschwohnung.de`

In [76]:
import requests
from bs4 import BeautifulSoup
import yaml
import os
import pandas as pd

# Setup

In [23]:
ROOT_URL = 'https://www.tauschwohnung.com'

## Login

In [9]:
LOGIN_URL = 'https://www.tauschwohnung.com/login'

cookies = {
    '_pk_id.1.3aa3': 'efce13a6140ddc5c.1683740292',
    'tauschwohnung': 'f5857706a5a85b28ec1a97a517ce8e478158582b',
    '_pk_ses.1.3aa3': '%2A',
    '_pk_cvar.1.3aa3': '%5B%5D',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/113.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'https://www.tauschwohnung.com',
    'Connection': 'keep-alive',
    'Referer': 'https://www.tauschwohnung.com/login',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'TE': 'trailers',
}

payload = yaml.safe_load('./credentials/tauschwohnung.yaml')

In [10]:
s = requests.session()
login_req = s.post(LOGIN_URL, headers=headers, cookies=cookies, data=payload)

if login_req.status_code != 200:
    raise ValueError('Login failed')

# Scraping

In [11]:
SEARCH_URL = 'https://www.tauschwohnung.com/search/result?city={city}&housing_type=1&rooms_min=1&storey_min=-1&storey_max=10&sort=standard&page={page}'

cities = [
    'Berlin',
    'Köln',
    'München',
    'Zürich',
    'Hamburg',
    'Wien',
    'Düsseldorf',
    'Frankfurt+am+Main',
    'Stuttgart'
]

## Stuttgart test

In [14]:
city = 'Stuttgart'

soup = BeautifulSoup(s.get(SEARCH_URL.format(city=city, page=1)).text, 'html.parser')

page_links = soup.find_all('a', class_ = 'page-link')
max_page = int(page_links[-2].text)


In [16]:
links = []
for page in range(1, 5):
    soup = BeautifulSoup(s.get(SEARCH_URL.format(city=city, page=page)).text, 'html.parser')
    for link in soup.find_all('a', class_ = 'stretched-link embed-responsive embed-responsive-4by3'):
        links.append(link['href'])

In [75]:
data = {
    'g-city': [],
    'g-rent': [],
    'g-surf': [],
    'g-rooms': [],
    't-city': [],
    't-max-rent': [],
    't-surf-min': [],
    't-rooms': []
}

for link in links:
    soup = BeautifulSoup(s.get(ROOT_URL + link).text, 'html.parser')
    strong = soup.find_all('strong')
    data['g-city'].append(city)
    data['g-rent'].append(strong[0].text)
    data['g-surf'].append(strong[1].text)
    data['g-rooms'].append(strong[2].text)
    data['t-city'].append(strong[5].text.strip())
    data['t-max-rent'].append(strong[7].text.strip())
    data['t-surf-min'].append(strong[8].text.strip())
    data['t-rooms'].append(strong[9].text.strip())

In [77]:
df = pd.DataFrame(data)

## Actual scraping

In [ ]:
data = {
    'g-city': [],
    'g-rent': [],
    'g-surf': [],
    'g-rooms': [],
    't-city': [],
    't-max-rent': [],
    't-surf-min': [],
    't-rooms': []
}

for city in cities:
    # Find how many pages we have to browse
    soup = BeautifulSoup(s.get(SEARCH_URL.format(city=city, page=1)).text, 'html.parser')
    page_links = soup.find_all('a', class_ = 'page-link')
    max_page = int(page_links[-2].text)

    # Store the links to browse
    links = []
    for page in range(1, max_page + 1):
        soup = BeautifulSoup(s.get(SEARCH_URL.format(city=city, page=page)).text, 'html.parser')
        for link in soup.find_all('a', class_ = 'stretched-link embed-responsive embed-responsive-4by3'):
            links.append(link['href'])

    # Browse the links and store data
    for link in links:
        soup = BeautifulSoup(s.get(ROOT_URL + link).text, 'html.parser')
        strong = soup.find_all('strong')
        data['g-city'].append(city)
        data['g-rent'].append(strong[0].text)
        data['g-surf'].append(strong[1].text)
        data['g-rooms'].append(strong[2].text)
        data['t-city'].append(strong[5].text.strip())
        data['t-max-rent'].append(strong[7].text.strip())
        data['t-surf-min'].append(strong[8].text.strip())
        data['t-rooms'].append(strong[9].text.strip())
        